In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

In [ ]:
from langchain.chat_models import init_chat_model
from enums.LLM import LLM

llm = init_chat_model(model=LLM.GPT4_1_MINI,temperature=0.4)
router_llm = init_chat_model(model=LLM.GPT4_1_MINI,temperature=0)

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType

expect_multi_question = "HP 포션좀 먹고 싶어, 물약좀 먹으면서 저 슬라임 공략법에 대해 알려줘"
expect_monster = "저거 공략법좀 알려줘"
expect_event = "이 석상을 부시면 어떻게 되는거야?"
expect_navigator= "현재 방에서 어디로 가야돼?"
expect_interaction_handler = "현재 방 불좀 켜줘"
expect_small_talk = "심심해"
expect_unknown_intent = "al;sdas"

question_prompt = PromptManager(FairyPromptType.FAIRY_ROUTER).get_prompt(question=expect_multi_question)
result = router_llm.invoke(question_prompt)

In [ ]:
result.content

'MULTI_ACTION_PLANNER'

In [ ]:
from agents.fairy.fairy_state import FairyRouteOutput, FairyState, FairyRouteType
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.types import Command
from agents.fairy.temp_string import reverse_questions
import random


def router(state: FairyState):
    last_message = None
    if isinstance(state["messages"][-1], HumanMessage):
        last_message = state["messages"][-1].content
    else:
        raise Exception("state.messages의 마지막 메시지가 휴면 메시지가 아닙니다.")

    parser_route_llm = router_llm.with_structured_output(FairyRouteOutput)
    route: FairyRouteOutput = parser_route_llm.invoke(last_message)

    if route.route == FairyRouteType.UNKNOWN_INTENT:
        clarification = reverse_questions[random.randint(0, 49)]
        return Command(
            goto="router",
            update={
                "messages": [AIMessage(clarification)], 
                "route_type": None
                },
        )
    return Command(goto="executor", update={"route_type": route.route})



def executor(state: FairyState):
    route = state.get("route_type")

    if route is None:
        raise Exception("executor 호출 전에 route_type이 설정되지 않았습니다.")

    # === ROUTE 분기 처리 ===
    if route == FairyRouteType.MONSTER_GUIDE:
        answer = ""

    elif route == FairyRouteType.EVENT_GUIDE:
        answer = ""

    elif route == FairyRouteType.DUNGEON_NAVIGATOR:
        answer = ""

    elif route == FairyRouteType.INTERACTION_HANDLER:
        answer = ""

    elif route == FairyRouteType.MULTI_ACTION_PLANNER:
        answer = ""

    elif route == FairyRouteType.SMALLTALK:
        answer = ""

    else:
        answer = (
            "음… 내가 이해하기 어려운 상황이야. "
            "혹시 다시 한 번 말해줄래?"
        )

    # === 응답 후 route_type 초기화 (중요!) ===
    return {
        "messages": [AIMessage(content=answer)],
        "route_type": None
    }


ImportError: cannot import name 'Command' from 'langgraph.graph' (/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/langgraph/graph/__init__.py)